These data came from duckdb document. This is a trail for window function.

In [1]:
import duckdb as ddb
import pandas as pd

conn = ddb.connect(database=':memory:')
conn.sql('CREATE TABLE power AS SELECT * FROM "power.csv"')
dates = conn.execute('SELECT DISTINCT "Date" FROM "power" ORDER BY "Date";').df()
ts_domain_dates = pd.date_range(start=dates['Date'].iloc[0], periods=dates.shape[0])
ts_domain_dates = ts_domain_dates.to_frame(name="t_date").reset_index(drop=True)
dates = pd.concat([dates, ts_domain_dates], axis=1)



In [3]:
conn.sql('CREATE OR REPLACE TABLE "t_date" AS SELECT CAST("Date" as Date) as "Date",CAST("t_date" as Date) as "t_date",  FROM "dates";')
rr = conn.execute('DESCRIBE power;').df()
rr

,column_name,column_type,null,key,default,extra
0,Plant,VARCHAR,YES,NaN,NaN,NaN
1,Date,DATE,YES,NaN,NaN,NaN
2,MWh,BIGINT,YES,NaN,NaN,NaN
3,BS,BIGINT,YES,NaN,NaN,NaN


In [6]:
rr = conn.execute('SELECT * FROM power;').df()
rr

,Plant,Date,MWh,BS
0,Boston,2019-01-03,507405.0,2.0
1,Boston,2019-01-04,528523.0,1.0
2,Boston,2019-01-05,469538.0,2.0
3,Boston,2019-01-06,474163.0,1.0
4,Boston,2019-01-07,507213.0,1.0
5,Boston,2019-01-09,582588.0,2.0
6,Boston,2019-01-10,499506.0,1.0
7,Boston,2019-01-11,482014.0,1.0
8,Boston,2019-01-12,486134.0,2.0
9,Boston,2019-01-13,531518.0,1.0


In [13]:
conn.execute(
    '''
    CREATE OR REPLACE TABLE "pre_data" AS SELECT * FROM "t_date" t LEFT JOIN "power" p USING ('Date') ORDER BY "t_date";
    '''
    )
rrr = conn.execute('SELECT * FROM "pre_data"').df()
rrrr = conn.execute('DESCRIBE pre_data').df()
rrr

,Date,t_date,Plant,MWh,BS
0,2019-01-02,2019-01-02,Worcester,118860.0,1.0
1,2019-01-03,2019-01-03,Worcester,101977.0,1.0
2,2019-01-03,2019-01-03,Boston,507405.0,2.0
3,2019-01-04,2019-01-04,Worcester,106054.0,1.0
4,2019-01-04,2019-01-04,Boston,528523.0,1.0
5,2019-01-05,2019-01-05,Worcester,92182.0,1.0
6,2019-01-05,2019-01-05,Boston,469538.0,2.0
7,2019-01-06,2019-01-06,Worcester,94492.0,2.0
8,2019-01-06,2019-01-06,Boston,474163.0,1.0
9,2019-01-07,2019-01-07,Boston,507213.0,1.0


In [21]:
df = conn.execute(
    '''
    SELECT "Plant", "Date","t_date","BS","MWh",
    SUM("MWh") OVER seven AS "MWh 3-day sum"
FROM "pre_data"
WINDOW seven AS (
    PARTITION BY "Plant", "BS"
    ORDER BY t_date ASC
    RANGE BETWEEN INTERVAL 0 DAYS PRECEDING
              AND INTERVAL 3 DAYS FOLLOWING)
ORDER BY 1, 3
    '''
).df()
df

,Plant,Date,t_date,BS,MWh,MWh 3-day sum
0,NaN,2019-01-15,2019-01-14,NaN,NaN,NaN
1,Boston,2019-01-03,2019-01-03,2.0,507405.0,976943.0
2,Boston,2019-01-04,2019-01-04,1.0,528523.0,1509899.0
3,Boston,2019-01-05,2019-01-05,2.0,469538.0,469538.0
4,Boston,2019-01-06,2019-01-06,1.0,474163.0,981376.0
5,Boston,2019-01-07,2019-01-07,1.0,507213.0,1006719.0
6,Boston,2019-01-09,2019-01-09,2.0,582588.0,1068722.0
7,Boston,2019-01-10,2019-01-10,1.0,499506.0,1513038.0
8,Boston,2019-01-11,2019-01-11,1.0,482014.0,1013532.0
9,Boston,2019-01-12,2019-01-12,2.0,486134.0,486134.0
